porco dio

In [1]:
import numpy as np  
import pandas as pd 

print("Hello, World!")

Hello, World!


/home/venpy/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
